# Data Overview

Input files are:
1) countries.txt - Static list of 198 countries downloaded from FAO. '0A - Food System POPULATION inputs 7-5-21.xlsx'
2) REF_supply.txt - REF farm gate food supply forecast by country and FAO commodity [kcal/(cap-day)]. '0C - Food System FOOD inputs 75 6-19-21.xslx'
3) food_lookup.txt - List of foods with categories and kcal/g values. '0C - Food System FOOD Inputs 75 6-19-21.xlsx'
4) waste_forecast.txt - Mass percentages of food losses and waste (in percentage of what enters each step) (Source: FAO 2011, Global Food Losses.) '0C - Food System FOOD Inputs 75 6-19-21.xlsx'
5) LCA_emissions_by_product.txt - Farm gate LCA impacts by FAO commodity, with max/min/avg values by study
6) population_high.txt - UN population estimates, high scenario.
7) population_medium.txt - UN population estimates, medium scenario.
8) REF_waste.txt - REF food waste forecast by country and FAO commodity [kcal/(cap-day)] '0C - Food System FOOD inputs 75 6-19-21.xslx'
9) plant_rich_diet.txt - Estimates of various diets' constituents (2/3_vegan, lacto_ovo_vegetarian, low_red_meat, meatless_day, no_dairy, no_red_meat, pescetarian, vegan)  '0C - Food System FOOD Inputs 75 6-19-21.xlsx'
10) food_waste_lookup.txt - List of foods included in the FAO commodity list, with various categories. '1 - Food System CORE 75 6-18-21.xlsx'



In [39]:
# Setup
import pandas as pd
from pathlib import Path
data_path = Path('./data/food/')

print('Reading data from', data_path.absolute())


# 'country_lookup' from '0A - Food System POPULATION inputs 7-5-21.xlsx' workbook:
# Static list of countries included in FAO analysis, with various categories used throughout the Food Systems model.
# header: "Country", "IPCC", "CountryExists", "Developed", "Demand region", "EU 28", "Diet region", "Waste region", "Waste region id"
countries = pd.read_csv(Path.joinpath(data_path,'countries.txt'), index_col='Country').convert_dtypes()

m = countries['CountryExists']==0
defunct_countries = countries[m].index.get_level_values(0).to_list()


Reading data from /mnt/c/Users/neilm/Documents/Drawdown/glass-wing/solutions/integrations/data/food


### REF supply, waste, and demand

### Overview
Food supply, waste, and demand in the reference scenario are based on the most recent comprehensive food supply data available and a recent global food demand forecast (Alexandratos et al. 2012). The original forecast applies only through 2050; however, we assume that trends from 2030-2050 continue until 2060.

The food demand forecast drives waste estimates and therefore supply estimates. This is in contrast to the original plant-rich diet and food waste models, which forecasted supply first and then derived waste based on those figures. By implication, we are assuming that global food demand is the ultimate driver of food production and therefore agricultural land use trends within the Food System.

### Implementation
The available baseline data provided by FAO is for supply rather than demand (i.e. consumption), and only runs through 2013 (as of January 2019). Further complicating things, FAO data specifically represent the amount of food supplied or ‘made available’ to consumers at the retail level. Adjustments are made early in the modeling process to account for this quirk of the data, to ensure that model assumptions and parameters relating to consumption, emissions, and waste are applied appropriately.

The Food System model begins in the FOOD inputs spreadsheet by adjusting raw ‘supply’ data from the FAO—which reflects the volume of food supplied to consumers—to instead reflect food supply at the farm-gate, i.e. as it leaves the farm and enters the downstream supply chain:

#### Fig. 1: Supply and waste flows in Food System model framework
![Image](./data/food/SupplyAndWasteFlows.png)

In the Drawdown Food System model framework, Demand is what remains after all losses and waste is accounted for, starting with the theoretical maximum supply.

From the diagram above:

	Sm = D + Wc + Ws + Wp

The LCA factors used in the model provide per-kcal emissions estimates based on the amount of food leaving the farm-gate. For this reason, “Supply” (S) in the model specifically refers to supply at the farm gate.

Similarly “Waste” refers to the sum ‘Wc + Ws’, or the total amount of food lost/wasted—i.e., not consumed—post–farm gate.

Since supply at the farm gate is the variable used in the model, we can isolate it in the framework:

	Supply = D + Wc + Ws = Sp + Ws

We can therefore apply the model’s waste factors to the FAO data (Sp) in order to back out upstream Supply values. The total amount of supply chain waste (Ws) can be calculated as a function of Supply using the pre-consumption, post–farm gate food loss rate (Ws’). The pre-consumption, post–farm gate food loss rate is derived by applying food loss rates from all included waste life-cycle stages:

	Ws’ = 1 - (1 – food handling and storage loss rate)*(1 – processing loss rate)*(1 – packaging loss rate)*(1 – distribution loss rate)

Supply chain waste can be calculated as:

	Ws = Supply * Ws’
	
The final calculation for Supply is therefore derived as follows, where the quantity (1-Ws’) represents the percentage of farm gate supply that ultimately reaches the consumer:

	Supply = Sp + Supply*Ws’
	Supply = Sp/(1-Ws’)



In [40]:
# 'REF_supply' from '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# REF farm gate food supply forecast by country and FAO commodity [kcal/(cap-day)]
REF_supply = pd.read_csv(Path.joinpath(data_path,'REF_supply.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()

# Use REF_supply as the master ['Country', 'FAO commodity'] index, updated with any missing values present in the plant_rich_diet dataset.
# All datasets using this index will have the same ['Country', 'FAO commodity'] combinations.

country_commodity_index = REF_supply.index.sort_values()
REF_supply = REF_supply.reindex(country_commodity_index)
REF_supply_kcal_g = REF_supply['kcal/g']

REF_supply.drop(['Sort code','Yield category', 'kcal/g'], axis='columns', inplace=True)

cols = REF_supply.columns.to_list()
years = [col for col in cols if len(col)==4 and col[:1]=='2']
years_int = [int(y) for y in years]

years_multi_index = pd.MultiIndex.from_product([['Years'],years])

REF_supply.columns = years_multi_index
  
# food_lookup file taken from the 'food_lookup' table on the 'food_lookup' tab in the '0C - Food System FOOD Inputs 75 6-19-21.xlsx' spreadsheet.
# 99 records.
# header: ["FAO commodity", "Waste category", "Diet category", "Demand category", "kcal/g", "Yield category"]
food_lookup  = pd.read_csv(Path.joinpath(data_path,'food_lookup.txt'), index_col=['FAO commodity']).convert_dtypes()

# 'waste_forecast' table from waste_forecast tab in '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# Mass percentages of food losses and waste (in percentage of what enters each step) (Source: FAO 2011, Global Food Losses.)
# waste_forecast data is by "Waste category" (e.g. Cereals, Milk and Egg) and "Waste Region" (e.g. Europe incl. Russia, North American and Oceania)
waste_forecast = pd.read_csv(Path.joinpath(data_path,'waste_forecast.txt'), index_col=['Waste category', 'Waste region']).convert_dtypes()


# Matches waste_forecast table on "Waste_forecast" tab in "1 - Food System CORE 75 6-18-21.xlsx".

# The box in the graphic above (Fig.1) "[handling, processing, packaging, distribution]" is disaggregated further into 
# (1-waste_forecast['Processing'])*(1-waste_forecast['Packaging']) in the line of code below.
# waste_forecast['Post-farm gate loss'] equals Ws' from the analysis above.

#	Ws’ = 1 - (1 – food handling and storage loss rate)*(1 – processing loss rate)*(1 – packaging loss rate)*(1 – distribution loss rate)
waste_forecast['Post-farm gate loss'] =  1 - (1-waste_forecast['Postharvest handling and storage'])*(1-waste_forecast['Processing'])*(1-waste_forecast['Packaging']) * \
                                        (1-waste_forecast['Distribution'])*(1-waste_forecast['Consumption'])


# Build a mappings table to make it easier to join datasets later.
mappings = country_commodity_index.to_frame()
mappings = mappings.join(countries['Waste region'])
mappings = mappings.join(food_lookup.loc[:,['Waste category','Yield category']])

waste_forecast = mappings.join(waste_forecast, on=['Waste category', 'Waste region'])
waste_forecast.drop(labels=waste_forecast.index.names, axis='columns', inplace=True)

tps = [['Data'],waste_forecast.columns]
waste_forecast.columns = pd.MultiIndex.from_product(tps) # Initialise multi-index with 'Other' grouping for non-year columns. Will add 'Years' later.

###

# "Pre-consumer, post-farm gate remains" (not used):
# waste_forecast['Pre-consumer'] =  (1-waste_forecast['Postharvest handling and storage'])*(1-waste_forecast['Processing'])*(1-waste_forecast['Packaging']) * \
#                                       (1-waste_forecast['Distribution'])


In [41]:
### Life-cycle analysis ###

## Farm gate LCA impacts by FAO commodity, with max/min/avg values by study. (g CO2e/kcal food) ##

# 'LCA range' taken from '0B - Food System LCA inputs 7-5-21.xslx' workbook.
#	Tilman & Clark; Heller & Keolian; Vieux et al.; Hoolohan et al.; Audsley et al. 
#	A few modifications were made in the development of the 2.0 integrated model, which modify previously documented methods: 
#	Several ‘oils’ were miscategorized as “oil crops” when assigning LCA impact factors from Tilman & Clark. Oils carry significantly higher impact factors (~2x with ‘min’ emission factors; ~3x with ‘avg’ factors;
#   and nearly 7x with ‘max’ factors).
#	“Rape and mustard seed” and “Palm kernels” were added to the FAO commodity list and assigned “oil crop” emission factors from Tilman & Clark. 
#	The emissions factor for “Vegetables, Other” reflects the appropriate category from Tilman & Clark, which has a material impact on the total system emissions due to significant supply/demand in this category.  

# header: "FAO commodity", "Sort code", "Diet category", "Min", "Avg", "Max", "kcal_per_gram". "Sort code" is a numeric index, which reflects the alphabetic order on FAO commodity (not used here).
LCA_emissions_by_product = pd.read_csv(Path.joinpath(data_path,'LCA_emissions_by_product.txt'), index_col='FAO commodity').convert_dtypes()

# Align with country_commodity_index to make it easier to join datasets later:
LCA_emissions_by_product = country_commodity_index.to_frame().join(LCA_emissions_by_product).drop(['Country','FAO commodity'],axis = 'columns')


In [42]:

# UN Population estimates from World Population Prospects, High Population Scenario, 2014-2100
# Taken from '0A - Food System POPULATION inputs 7-5-21.xlsx', pop_REF1 tab.
population_high = pd.read_csv(Path.joinpath(data_path,'population_high.txt'), index_col='Country').convert_dtypes()
population_high.columns = years_multi_index

# UN Population estimates from World Population Prospects, Medium Population Scenario (reflecting "family planning" scenario intervention), 2014-2100
# Taken from '0A - Food System POPULATION inputs 7-5-21.xlsx', pop_REF2 tab.
population_medium = pd.read_csv(Path.joinpath(data_path,'population_medium.txt'), index_col='Country').convert_dtypes()
population_medium.columns = years_multi_index

#Align to the standard index for easier joining:
population_commodity_high = country_commodity_index.to_frame().join(population_high['Years']).drop(['Country','FAO commodity'],axis = 'columns')
population_commodity_high.columns = years_multi_index

population_commodity_medium = country_commodity_index.to_frame().join(population_medium['Years']).drop(['Country','FAO commodity'],axis = 'columns')
population_commodity_medium.columns = years_multi_index


The model then estimates supply in 2014 for each FAO commodity in each country, forecasting growth into the model’s first year (2015): 

    Food supply(2014) = Average(FAO_supply[2011:2013]) 

    REF_supply(2015) = Food supply(2014) * (1 + Alexandratos growth factor [Ag])

Supply is then converted to waste using the post–farm gate food loss rate by waste category and waste region (Wf), which is assumed to be constant in the REF case:

    REF_waste(2015) = REF_supply(2015)*Wf

The source of food loss rate data does not provide estimates for all foods; those not covered explicitly are grouped together under an “Other” waste category. To calculate post–farm gate food loss rates for this “Other” category, the waste system model is run, for all other waste categories through to its end in order to generate waste mass by food life cycle stage. Using these final waste mass values, the “Other” post–farm gate food loss rates are estimated as the total waste by region divided by the total supply by region. 

This produces the REF_supply and the REF_waste datasets, which are saved from the spreadsheet and read in by the code below.


In [43]:
# 'REF_waste' table from REF_waste tab in '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# REF food waste forecast by country and FAO commodity [kcal/(cap-day)]
REF_waste = pd.read_csv(Path.joinpath(data_path,'REF_waste.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()
REF_waste.drop(['Sort code', 'Waste category'], axis='columns', inplace=True)
REF_waste = REF_waste.reindex(country_commodity_index)
REF_waste.columns = years_multi_index


These two values (waste and supply) are then converted to demand, which can be thought of as the portion of food supply that is not wasted. In other words: 

    REF_demand(2015) = REF_supply(2015) - REF_waste(2015)

With this first year’s demand defined, the process order flips, and demand becomes the baseline. We express demand in a given year (from 2016-2060) as a function of the previous year’s demand (rather than as an exponential function of time since 2015), because Ag factors are different stepwise from 2015-2030, and from 2030-2060: 

    REF_demand(Year) = REF_demand(Year - 1) * (1 + Ag)

Because of the simple algebraic relationship between demand, waste, and supply, we can express waste as a function of only demand and the post–farm gate loss rate (Wf): 

	Demand + Waste = Supply; Waste = Supply * Wf
	Demand + Waste = Waste/Wf
	Waste/Wf - Waste = Demand
	(Waste - Waste*Wf)/Wf = Demand
	Waste * (1-Wf)/Wf = Demand
	Waste = Demand * Wf/(1-Wf)

The REF waste forecast for each year from 2016-2060 is thus generated according to the equation: 

	REF_waste = REF_demand * Wf/(1-Wf)
The REF supply forecast for each year from 2016-2060 is then completed by taking the sum of demand and waste: 

    REF_supply = REF_demand + REF_waste


In [44]:
# As described above, REF_demand is calculated like this:
REF_demand = REF_supply - REF_waste


### Plant-rich demand

Plant-rich demand is generated on a per-capita basis, as are the REF forecasts, and can be conceptualized as the “average diet of each person who adopts a plant-rich diet”. This is also best conceptualized as a model input, not a forecast per se, since its purpose is to feed into the PDS demand forecast rather than to represent (as it technically does) what food demand consumption would look like if 100 percent of people adopted a plant-rich diet starting in 2015. 
The ‘plant-rich diet’ as modeled in the FOOD inputs spreadsheet is a combination of:

 A. the “healthy” diet as conceived by Bajzelj et al. (2014) for various food groups (‘diet categories’),

 B. the assumed total caloric demand (a key input parameter in the Food System model), and 
 
 C. the estimated balance of demand across FAO commodities in each given year.

C is used to allocate demand among FAO commodities within the diet categories from Bajzelj et al. on the basis of relative forecasted demand among those FAO commodities. This is expressed as a ‘relative demand factor’ (Df), expressing the percentage of diet category calories comprised of each given FAO commodity.
For each FAO commodity, in a given country, in a given year: 

    Df = REF_demand(commodity, country, year)/SUM over diet category{REF_demand(diet category, country, year)} 

This factor is multiplied by the ‘healthy’ diet demand from Bajzelj et al., then normalized to total calories in the plant-rich diet (Ci), to yield the plant-rich demand for each FAO commodity, in each country, in each year: 

    Plantrich_demand = Df(FAO commodity, country, year) * ‘Healthy’ demand(diet region, diet category)*Ci/ SUM{‘healthy’ diet calories in given region}
    
As long as the total daily caloric consumption input parameter for plant-rich diet is set to 2500 or less, this approach conforms with the Bajzelj et al. framework’s “maximum” limits on the consumption of certain commodity groups, for example including beef and vegetable oils. If that total daily caloric consumption input value is less than 2500 calories, however, then the resulting caloric demands for commodity groups at or near the Bajzelj et al. framework’s “minimum” limits will fall below those limits. This analysis therefore implicitly assumes that these “minimum” floors are designed to assign a minimum relative concentration of fruits and vegetables in individuals’ diet, rather than a minimum absolute concentration. This assumption should be revisited if/when the dietary model is improved in the future (perhaps also to update assumptions about minimum protein requirements, for example), as it does not appear to be in keeping with Bajzelj et al.’s intent. 

NB: in the first draft of the v2 model, a short list of countries were under-assigned calories in their plant-rich diets due to those countries not having a baseline demand for FAO commodities in a given diet category. Without a baseline REF_demand (e.g., zero “maize” demand in 2011-2013), in the above formulae “Df” would be zero, and thus that category would be assigned zero calories and the total calories in the resulting plant-rich diet would be less than the target value set in the scenario parameters.

This was addressed by: 
1.	Creating new entries in the core database for several unique country-commodity pairs, increasing the total number of entries from 10,687 to 10,713. Countries had been identified as missing foods from a given “diet category”—not specific FAO commodities. Rather than assuming an arbitrary distribution of commodities, discretion was used to choose single FAO commodities in each ‘missing’ category for each affected country. Commodities were chosen to be as general as possible—“Pulses, Other and products” for the pulses category; “Maize and products” for the maize category; “Marine Fish, Other” for the fish category; and “Cereals, Other” for the other grains category. 
2.	Each of these added country-commodity pairs—and two others, which were already in the database but with zero-values—are then assigned very low supply (and therefore waste and demand) values of 0.001 kcal/cap-day in 2015. This low value is chosen to ensure that their impact on the REF forecast is negligible—though in the above formulae any non-zero value will have a larger impact on the final plant-rich diets in the affected countries. 
An alternative approach would have been to merely assign the ‘missing’ calories in proportion to all existing food commodities. This approach may be slightly more complex to implement, but is worthy of consideration in future model iterations. 




In [45]:

parameters = {
    'localization_emisssion_reduction_percentage': 0.05,
    'total_food_waste_reduction': 0.75,
    'diet_start_year': 2019,
    'diet_end_year': 2050,
    'waste_start_year': 2019,
    'waste_end_year': 2050,
    'plant_rich_diet_adoption': 0.75,
    'plant_rich_diet_kcal_per_day': 2300
    }

# plant_rich_diet taken from the 'plant-rich_diet' tab on the '0C - Food System FOOD Inputs 75 6-19-21.xlsx' spreadsheet.
# The country field contains the following country names, which result in failed joins: "C√É¬¥te d'Ivoire", "Czechia". These were corrected.
# fbs = Food Balance Sheet
# columns: [country, fbs_item, 2/3_vegan, lacto_ovo_vegetarian, low_red_meat, meatless_day, no_dairy, no_red_meat, pescetarian, vegan, GrandTotal]

# plant_rich_diet reindexed to country_commodity_index. Then it will have the same number of rows and ['Country', 'FAO commodity'] combinations as the other datasets.
plant_rich_diet = pd.read_csv(Path.joinpath(data_path,'plant_rich_diet.txt'), index_col=['Country', 'FAO commodity'])
plant_rich_diet = plant_rich_diet.reindex(country_commodity_index, fill_value = 0.0)

plantrich_demand = plant_rich_diet.join(other=food_lookup[['Diet category','Waste category']], how='inner').join(countries[['Diet region','Waste region']])[['GrandTotal', 'Diet category', 'Waste category', 'Diet region', 'Waste region']]
plantrich_demand['GrandTotal'] = plantrich_demand['GrandTotal'] * parameters['plant_rich_diet_kcal_per_day'] / 2300



In [46]:

# @IF(AND(NUMBERVALUE(waste_forecast[[#Headers],[2014]])>=parameters!$E$6,NUMBERVALUE(waste_forecast[[#Headers],[2014]])<=parameters!$F$6),
#        @waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]*(1-parameters!$A$7*(NUMBERVALUE(waste_forecast[[#Headers],[2014]])-parameters!$E$6+1)/(parameters!$F$6-parameters!$E$6+1)),
#       IF(NUMBERVALUE(waste_forecast[[#Headers],[2014]])>parameters!$F$6,@waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]*(1-parameters!$A$7),waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]))

start_idx = years_int.index(parameters['waste_start_year'])
end_idx = years_int.index(parameters['waste_end_year'])

for y in years_int[:start_idx]:
    waste_forecast[('Years',str(y))] = waste_forecast[('Data', 'Post-farm gate loss')]

for y in years_int[start_idx:end_idx]:
    waste_forecast[('Years',str(y))] = waste_forecast[('Data', 'Post-farm gate loss')] * (1-parameters['total_food_waste_reduction'] * (y - parameters['waste_start_year'] + 1 )/ (parameters['waste_end_year'] - parameters['waste_start_year'] + 1))

for y in years_int[end_idx:]:
    waste_forecast[('Years',str(y))] = waste_forecast[('Data', 'Post-farm gate loss')] * (1-parameters['total_food_waste_reduction'])

# From food_lookup tab in '1 - Food System CORE 75 6-18-21.xlsx'
# FAO commodity, Waste category, Diet cateory, Demand category, kcal/g, Yield category:
# List of foods included in the FAO commodity list, with various categories used throughout the Food Systems model.
food_waste_lookup = pd.read_csv(Path.joinpath(data_path,'food_waste_lookup.txt'), index_col=['FAO commodity']).convert_dtypes()



### Adoption
Starting in the CORE spreadsheet, attention shifts to the Project Drawdown Scenario (PDS) forecasts, which are driven by the adoption assumptions for both plant-rich diet and food waste.

Adoption for both solutions grows linearly, starting in the first year of adoption and with adoption reaching its target value in the final year. Both the first and final year of adoption are separate input parameters for each solution. So, for each solution, we have adoption as a function of the current year (Y), adoption start year (Yo), and adoption completion year (Yf):

    Adoption rate (A) = A*(Y-Yo+1)/(Yf-Yo+1)
    
This function is designed to have the properties that in Yf, adoption = A; in year (Yo - 1), adoption is zero, and that in the intermediate years adoption grows linearly. If the start year is later than 2015, adoption is assumed to be zero until the first year; if the completion year is earlier than 2060, then adoption rate is held constant (at A) for the remainder of the forecast.



In [47]:

adoption_rate_list = []
start_yr, end_yr, adoption = parameters['diet_start_year'], parameters['diet_end_year'], parameters['plant_rich_diet_adoption']
denominator = end_yr - start_yr + 1

for y in range(years_int[0],start_yr):
    adoption_rate_list.insert(0, 0.0)

for y in range(start_yr, end_yr+1):
    val =  adoption * (y - start_yr +1) / denominator
    adoption_rate_list.append(val)

for y in range(end_yr, years_int[-1]):
    adoption_rate_list.append(adoption)

# Represent the adoption rates by a matrix to make it easier to multiply across datasets:
adoption_rates = pd.DataFrame.from_dict(data={'Years': adoption_rate_list}, orient='index', columns=years)
adoption_rate_frame = country_commodity_index.to_frame().join(adoption_rates, how='cross').set_index(country_commodity_index).drop(labels=['Country', 'FAO commodity'], axis='columns')
adoption_rate_frame.columns = years_multi_index


### DIET demand, waste, and supply

The DIET forecasts for demand, waste, and supply in the CORE spreadsheet flow from demand forecasts, mirroring the REF forecast methodology starting in 2016. “DIET_demand” values are constructed based on adoption to-date of the plant-rich diet solution: 

    DIET_demand = Plantrich_demand * A + REF_demand * (1 - A)
As in the REF forecast from 2016 onwards, PDS_waste is simply a function of PDS_demand and each given year’s waste factor, which remains constant: 

    DIET_waste = DIET_demand * Wf/(1-Wf)
DIET_supply is simply forecast as the sum of DIET_demand and DIET_waste: 

    DIET_supply = DIET_demand + DIET_waste


In [48]:
# Build DIET_supply (this cell) and DIET_plant_supply (later) to match the two tables on the DIET_supply tab from "1 - Food System CORE 75 6-18-21.xlsx".

# DIET_supply: Per-capita DIET-only total food supply forecast, 2014-2100 [kcal/(cap-day)]
# DIET_plant_supply: DIET food supply forecast *plant-rich diet only*, 2014-2100 [kcal/(cap-day)]

# Implements "DIET_demand = Plantrich_demand * A + REF_demand * (1 - A)":

grand_total = plantrich_demand['GrandTotal'].to_frame()
grand_total.columns = ['Years']
DIET_demand = grand_total.multiply(adoption_rate_frame, axis='columns', level=0) + REF_demand.multiply(1-adoption_rate_frame, level=1) # use level=0 in the first term to broadcast the multiplication across all years.
# DIET_demand now matches the DIET_demand table on the spreadsheet.

# DIET_demand[@2014]*VLOOKUP(@DIET_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[#All],11,0)/(1-VLOOKUP(@DIET_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[#All],11,0))
# Implements "DIET_waste = DIET_demand * Wf/(1-Wf)":

post_farm_gate = waste_forecast['Data']['Post-farm gate loss'].map(lambda x: x/(1-x)).to_frame()
post_farm_gate.columns = ['Years']
DIET_waste = post_farm_gate.multiply(DIET_demand, axis='columns', level=0)

DIET_supply = DIET_demand + DIET_waste



In [49]:
# reg test for DIET_supply (and implicity DIET_demand, DIET_waste, REF_demand):
# Takes 18m 33s to run.

# from math import isclose

# dd_reg = pd.read_csv(Path.joinpath(data_path,'regression_test_DIET_food_supply_core.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()
# #dd_reg.index = dd_reg.index.sort_values()

# invalid_countries = {'Côte d\'Ivoire','Czech Republic'}

# test_countries = set(DIET_supply.index.get_level_values(0)).difference(invalid_countries)
# test_categories = set(dd_reg.index.get_level_values(1))

# for y in years:
#     print(y,end='|')
#     for c in test_countries:
#         #print(c,end='|')
#         s = dd_reg.index.get_loc(c)
#         for cat in dd_reg.iloc[s].index.get_level_values(1):
#             try:
#                 left = DIET_supply.loc[(c,cat),y]
#                 right = dd_reg.loc[(c,cat),y]
#                 if not isclose(left, right, rel_tol=1e-07):
#                     raise ValueError(f'{left=}, {right=}')
#             except Exception as ex:
#                 print(f'failed for year {y=}, country {c=}, category {cat=}')
#                 print(f'DIET_supply value: {left=}, dd_reg value:{right=}')
            


### WASTE demand, waste, and supply
The WASTE forecasts for demand, waste, and supply in the CORE spreadsheet flow from demand forecasts, mirroring the REF forecast methodology starting in 2016. “WASTE_demand” values are unchanged from the REF case: 

    WASTE_demand = REF_demand

As in the REF forecast, WASTE_waste is simply a function of WASTE_demand and each given year’s waste factor. In this case, however, the waste factor (Wf) decreases over time as the adoption of the food waste reduction solution increases: 

    WASTE_waste = WASTE_demand * Wf/(1-Wf)

WASTE_supply is simply forecast as the sum of WASTE_demand and WASTE_waste: 

    WASTE_supply = WASTE_demand + WASTE_waste


In [50]:
# Per-capita WASTE-only total food supply forecast, 2014-2100 [kcal/(cap-day)]
# WASTE_supply from "1 - Food System CORE 75 6-18-21.xlsx".

# formula e.g. =SUM(WASTE_waste[@2017],WASTE_demand[@2017])
# waste_supply = pd.read_csv(r'/mnt/c/Users/neilm/Documents/Drawdown/Excel Files/food_system/data/WASTE_supply.txt', index_col=['Country', 'FAO commodity'])

# Need to calculate WASTE_waste and WASTE_demand # Note: WASTE_demand = REF_demand in the spreadsheet
# To calculate WASTE_waste:
# WASTE_demand[@2014]*INDEX(waste_forecast[#All],MATCH(@WASTE_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(WASTE_waste[[#Headers],[2014]],waste_forecast[#Headers],0))/(1-INDEX(waste_forecast[#All],MATCH(@WASTE_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(WASTE_waste[[#Headers],[2014]],waste_forecast[#Headers],0)))
waste_forecast_years = waste_forecast.drop('Data', axis='columns')

WASTE_waste = REF_demand.multiply(waste_forecast_years).divide(1-waste_forecast_years)

WASTE_supply = REF_demand + WASTE_waste


### PDS demand, waste, and supply
The Project Drawdown Scenario (PDS) forecasts for demand, waste, and supply in the CORE spreadsheet flow from demand forecasts, mirroring the REF forecasts starting in 2016. As in the DIET case, “PDS_demand” values are constructed based on adoption to-date of the plant-rich diet solution: 

    PDS_demand = Plantrich_demand * A + REF_demand * (1 - A)

As in the WASTE case, the waste factor (Wf) decreases over time as the adoption of the food waste reduction solution increases: 

    PDS_waste = PDS_demand * Wf/(1-Wf)

PDS_supply is simply forecast as the sum of PDS_demand and PDS_waste: 

    PDS_supply = PDS_demand + PDS_waste


In [51]:
# Per-capita PDS total food supply forecast, 2014-2100 [kcal/(cap-day)]
# PDS_supply from "1 - Food System CORE 75 6-18-21.xlsx".

# PDS_demand = Plantrich_demand * A + REF_demand * (1 - A)
PDS_demand = plantrich_demand['GrandTotal'].rename('Years').to_frame().multiply(adoption_rate_frame, axis='columns', level=0)
PDS_demand = PDS_demand + REF_demand.multiply(1-adoption_rate_frame)

# PDS_waste = PDS_demand * Wf/(1-Wf)
PDS_waste = PDS_demand.multiply(waste_forecast_years).divide(1-waste_forecast_years)
PDS_supply = PDS_demand + PDS_waste


In [52]:

# Build DIET_plant_supply:
post_farm_gate = waste_forecast[('Data','Post-farm gate loss')].to_frame()
post_farm_gate.columns = ['Years']

# Plant-rich_supply = Plant-rich_demand * A / (1-Wf)
plantrich_demand_adoption = plantrich_demand['GrandTotal'].rename('Years').to_frame().multiply(adoption_rate_frame, level=0)
DIET_plant_supply = plantrich_demand_adoption.divide(1-post_farm_gate, level=0)

# Then PDS_plant_supply:

# PDS food supply forecast *plant-rich diet only*, 2014-2100 [kcal/(cap-day)]
# PDS_plant_supply table on the PDS_supply tab in the "1 - Food System CORE 75 6-18-21.xlsx" workbook.
# formula e.g. PDS_plant_supply = plantrich_demand[@2017]*PDS_demand!G$2/(1-INDEX(waste_forecast[#All],MATCH(@PDS_plant_supply[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(PDS_plant_supply[[#Headers],[2017]],waste_forecast[#Headers],0)))

# PDS_plant_supply = pds_supply_plant_rich comes from plantrich_demand, which comes from plant_rich_diet, which is read in from a file.

PDS_plant_supply = plantrich_demand_adoption.divide(1-waste_forecast_years)

# PDS_plant_supply now matches PDS_plant_supply table on the PDS_supply tab in the "1 - Food System CORE 75 6-18-21.xlsx" workbook.


In [53]:
# reg test for PDS_supply (and implicitly PDS_demand, REF_demand, plantrich_demand, PDS_waste):
# from math import isclose

# dd_reg = pd.read_csv(Path.joinpath(data_path,'regression_test_PDS_food_supply.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()

# invalid_countries = {'Côte d\'Ivoire','Czech Republic'}

# test_countries = set(PDS_supply.index.get_level_values(0)).difference(invalid_countries)
# test_categories = set(dd_reg.index.get_level_values(1))

# for y in years:
#     print(y,end='|')
#     for c in test_countries:
#         #print(c,end='|')
#         s = dd_reg.index.get_loc(c)
#         for cat in dd_reg.iloc[s].index.get_level_values(1):
#             try:
#                 left = PDS_supply['Years'].loc[(c,cat),y]
#                 right = dd_reg.loc[(c,cat),y]
#                 if not isclose(left, right, rel_tol=1e-07):
#                     raise ValueError(f'{left=}, {right=}')
#             except Exception as ex:
#                 print(f'failed for year {y=}, country {c=}, category {cat=}')
#                 print(f'PDS_supply value: {left=}, dd_reg value:{right=}')
            

### PDS and REF emissions
While demand drives waste and supply in the Food System model, emissions—calculated in the EMISSIONS and YIELD spreadsheet—still result from the total supply of food. In the REF case, this is simply expressed as the per-capita, per-day supply multiplied by LCA emissions impact factor for each food commodity, then multiplied by 365 and divided by 10^6 (g/tonne conversion) to generate annual values in tonnes for each given country in a given year. This calculation also applies to the WASTE scenario: 

    REF_emissions = SUM over all commodities {REF_supply * If * Pop} * 365 / 10^6

Emissions in the PDS case are slightly more complex, because in the modeling framework we have part of the population who is eating the REF case diet (1-A), and another part that is eating the plant-rich diet (A) in any given year. The impact of plant-rich diet also includes a ‘localization factor’ (Lf) that is assumed to reduce emissions by 5 percent across FAO commodities, which is applied as a reduction to the LCA impact factor (If). For this reason, we need to isolate the per-capita supply of plant-rich diet. This is generated by defining plant-rich supply as a function of plant-rich demand (using the same algebra used previously to calculate waste as a function of demand). These calculations also apply to the DIET scenario: 

	Demand + Waste = Supply
	Waste = Supply * Wf
	Supply = Demand + Supply * Wf
	Supply * (1 - Wf) = Demand
	Supply = Demand/(1 - Wf)
Since we are only interested in plant-rich supply, we must adjust for adoption: 

	Plant-rich_supply = Plant-rich_demand*A/(1-Wf)
We can then express the total PDS emissions as the sum of emissions from its component parts: plant-rich diet supply for the fraction of people who have ‘adopted’ the solution, and REF diet supply for the fraction who have not, in a given year: 

    PDS emissions = [REF_supply*If*Pop*(1-A) + Plant-rich_supply*If*(1- Lf)*Pop*A]*365/10^6


In [54]:
# REF Emissions

# Implements: "REF_emissions = SUM over all commodities {REF_supply * If * Pop} * 365 / 10^6"
lca_emmissions_avg = LCA_emissions_by_product['avg'].rename('Years').to_frame()

REF_supply_population_high = REF_supply.multiply(population_commodity_high) # REF_supply read in from file.
REF1_emissions = REF_supply_population_high.multiply(lca_emmissions_avg * 365 / 10**6, level=0) # multiplies each column (year) by avg, matching by commodity.

REF_supply_population_medium = REF_supply.multiply(population_commodity_medium)
REF2_emissions = REF_supply_population_medium.multiply(lca_emmissions_avg * 365 / 10**6, level=0)
    

In [55]:
# PDS Emissions


# Switch spreadsheets here to "Food System Emissions and Yield 75 6-20-21.xls"
# Calculate DIET_emissions:
diet_diff = DIET_supply - DIET_plant_supply
avg = LCA_emissions_by_product['avg'].rename('Years').to_frame()
diet_diff_by_product = diet_diff.multiply(avg, level=0, fill_value=0.0)
diet_diff_by_product_population = diet_diff_by_product.multiply(population_medium) 

diet_plant_supply_by_product = DIET_plant_supply.multiply(avg, level=0, fill_value=0.0)
diet_plant_supply_by_product_population = diet_plant_supply_by_product.multiply(population_medium)
diet_plant_supply_by_product_population = diet_plant_supply_by_product_population * (1-parameters['localization_emisssion_reduction_percentage'])

diet_emissions = (diet_diff_by_product_population + diet_plant_supply_by_product_population) * 365/10**6


##############################################

# calculate waste emissions:
waste_supply_by_product = WASTE_supply.multiply(avg, level=0)
waste_emissions = waste_supply_by_product.multiply(population_medium) * 365/10**6

# calculate PDS emissions:

pds_diff = PDS_supply - PDS_plant_supply
pds_diff_by_product = pds_diff.multiply(avg, level=0)
pds_diff_by_product_population = pds_diff_by_product.multiply(population_medium) 

pds_plant_supply_by_product = PDS_plant_supply.multiply(avg, level=0)
pds_plant_supply_by_product_population = pds_plant_supply_by_product.multiply(population_medium)
pds_plant_supply_by_product_population = pds_plant_supply_by_product_population * (1-parameters['localization_emisssion_reduction_percentage'])

pds_emissions = (pds_diff_by_product_population + pds_plant_supply_by_product_population) * 365/10**6


In [56]:
# regression test for diet_annual_emissions, and implicitly: PDS_plant_supply, diet_supply:

# from math import isclose

# diet_emissions_file = pd.read_csv(Path.joinpath(data_path,'regression_test_DIET_emissions.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()

# #invalid_countries = {'Côte d\'Ivoire','Czech Republic', 'China, Macao SAR', 'China, Hong Kong SAR'}
# invalid_countries = {'Côte d\'Ivoire','Czech Republic'}

# test_countries = set(diet_emissions.index.get_level_values(0)).difference(invalid_countries.union(defunct_countries))
# test_categories = set(diet_emissions_file.index.get_level_values(1))

# for y in years:
#     print(y,end='|')
#     for c in test_countries:
#         #print(c,end='|')
#         s = diet_emissions_file.index.get_loc(c)
#         for cat in diet_emissions_file.iloc[s].index.get_level_values(1):
#             try:
#                 left = diet_emissions['Years'].loc[(c,cat),y]
#                 right = diet_emissions_file.loc[(c,cat),y]
                
#                 if not isclose(left, right, rel_tol=1e-05):
#                     raise ValueError(f'{left=}, {right=}')
#             except Exception as ex:
#                 print(f'failed for year {y=}, country {c=}, category {cat=}')
#                 print(f'diet_emissions value: {left=}, diet_emissions_file value:{right=}')


In [57]:
# Reporting tables: REF1_country_per_capita_emissions, REF2_country_per_capita_emissions

REF1_country_emissions = REF1_emissions.groupby(level='Country').sum()
REF1_country_per_capita_emissions = REF1_country_emissions.divide(population_high *365 / (10**3), fill_value=0.0)

REF2_country_emissions = REF2_emissions.groupby(level='Country').sum()
REF2_country_per_capita_emissions = REF2_country_emissions.divide(population_medium * 365 / (10**3), fill_value=0.0)


In [58]:
# Reporting tables: pds_diet_per_capita_emissions, waste_per_capita_emissions, pds_per_capita_emissions

# Note: [NL, 14 Oct 2021], the following three tables have incorrect formulas in the spreadsheet:
# pds_diet_per_capita_emissions, waste_per_capita_emissions, pds_per_capita_emissions

pds_diet_country_emissions = diet_emissions.groupby(level='Country').sum()
pds_diet_per_capita_emissions = pds_diet_country_emissions.divide(population_medium * 365 / (10**3), fill_value=0.0)

waste_country_emissions = waste_emissions.groupby(level='Country').sum()
waste_per_capita_emissions = waste_country_emissions.divide(population_medium * 365 / (10**3), fill_value=0.0)

pds_country_emissions = pds_emissions.groupby(level='Country').sum()
pds_per_capita_emissions = pds_country_emissions.divide(population_medium * 365 / (10**3), fill_value=0.0)



In [59]:
# Reporting tables: REF1_commodity_CO2, REF2_commodity_CO2, diet_commodity_CO2, waste_commodity_CO2, pds_commodity_CO2

REF1_commodity_CO2 = REF1_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
REF2_commodity_CO2 = REF2_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.

diet_commodity_CO2 = diet_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
waste_commodity_CO2 = waste_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
pds_commodity_CO2 = pds_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.

### Global food yield
Global food yield is generated in the EMISSIONS and YIELD spreadsheet by a simple calculation—summing total global food supply in the REF and PDS scenarios within the food categories used in the Yield model (‘yield categories’). 
Yield categories include “Grains” and individual livestock-related categories. “Grain” totals are the sum of the following categories, used in the food waste model: ‘Cereals’, ‘Fruits and Vegetables’, ‘Oilseeds and pulses’, ‘Roots and tubers’, and ‘Other’. The addition of the “Other” category, whey, and fish/seafood entries are changes from the first version of the food system model. Livestock-related yield categories (and fish/seafood entries) reflect the totals of their own individual FAO commodities. 
Global yield for each category in each given year is calculated in grams after applying a kcal-to-g conversion factor as: 

    Yield = SUM, over all FAO commodities and countries { Supply * g/kcal * Pop} * 365

NB: the kcal-to-g conversion factor was generated by downloading 2013 Food Supply tables for both kcal/capita/day and g/capita/day and dividing the former by the latter for each FAO commodity. For commodities that either (1) are included in FAO supply tables but with zero supply indicated, or (2) are included in FAO supply tables [generally with only a few countries reporting] but show up in the global average as having zero average caloric demand, the following assumptions have been made: 
-	Sugar Beet is assumed to have an equal caloric density as Sugar (Raw equivalent)
-	Caloric density values for Cloves (“cloves, ground”, in the USDA database), Pepper (“pepper, black”), and Rape and Mustardseed (“mustardseed, ground”) are taken from the USDA nutritional database, as they are estimated as zero values when derived directly from FAO data, which leads to calculation errors. 
-	The caloric density value for Palm Kernels is assumed to be the average of Palm Oil and Palmkernel Oil, a decision currently without strong justification but also with limited impact on model results given how small Palm Kernels’ supply is. 
-	The values for Molasses; Meat, Aquatic Mammals; Whey; and Roots & Tuber Dry Equiv are all left as zeros because they all have zero supply in the FAO tables anyway
-	The values for Fish, Body Oil; and Fish, Liver Oil are assumed to contain 9 kcal/g on the basis of being approximately pure fat
-	The value for Aquatic Animals, Others is calculated as a simple average of the kcal/g factors for: Cephalopods; Crustaceans; Demersal Fish; Freshwater Fish; Marine Fish, Other; Molluscs, Other; and Pelagic Fish

The resulting yield forecast values are divided by 10^12 to convert from g to million-metric-tonnes. 


In [60]:
# Global food yield

waste_supply_year_yield_pop_ref2 = WASTE_supply.copy() # old: waste_supply
pds_supply_year_yield_pop_ref2 = PDS_supply.copy() # old: pds_supply_core

kcal_per_gram = REF_supply_kcal_g.rename('Years').to_frame()

REF_supply_year_yield = REF_supply.divide(kcal_per_gram, level=0)
REF_supply_year_yield_pop_ref1 = REF_supply_year_yield.multiply(population_high * 365, fill_value=0.0)
REF_supply_year_yield_pop_ref2 = REF_supply_year_yield.multiply(population_medium * 365, fill_value=0.0)

diet_supply_year_yield = DIET_supply.divide(kcal_per_gram, level=0)
diet_supply_year_yield_pop_ref2 = diet_supply_year_yield.multiply(population_medium * 365, fill_value=0.0)

waste_supply_year_yield = WASTE_supply.divide(kcal_per_gram, level=0)
waste_supply_year_yield_pop_ref2 = waste_supply_year_yield.multiply(population_medium * 365, fill_value=0.0)

pds_supply_year_yield = PDS_supply.divide(kcal_per_gram, level=0)
pds_supply_year_yield_pop_ref2 = pds_supply_year_yield.multiply(population_medium * 365, fill_value=0.0)


report_dict = {'Grain': {'Cereals', 'Fruits and vegetables', 'Oilseeds and pulses', 'Roots and tubers', 'Other'},
               'Meat': {'Bovine Meat', 'Meat, Other', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Fats, Animals, Raw', 'Offals, Edible'},
               'Milk and egg': {'Butter, Ghee', 'Cream', 'Eggs', 'Milk - Excluding Butter', 'Whey'},
               'Fish and seafood': {'Aquatic Animals, Others', 'Aquatic Plants', 'Cephalopods', 'Crustaceans', 'Demersal Fish', 'Fish, Body Oil', 'Fish, Liver Oil',
                                    'Freshwater Fish', 'Marine Fish, Other',  'Meat, Aquatic Mammals', 'Molluscs, Other', 'Pelagic Fish'}
               }

#REF_supply_year_yield_pop_ref1_yc = REF_supply_year_yield_pop_ref1.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
REF_supply_year_yield_pop_ref1_yc = REF_supply_year_yield_pop_ref1['Years'].join(mappings['Yield category']).groupby('Yield category').sum()
REF_supply_year_yield_pop_ref1_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

#REF_supply_year_yield_pop_ref2_yc = REF_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
REF_supply_year_yield_pop_ref2_yc = REF_supply_year_yield_pop_ref2['Years'].join(mappings['Yield category']).groupby('Yield category').sum()
REF_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

#diet_supply_year_yield_pop_ref2_yc = diet_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
diet_supply_year_yield_pop_ref2_yc = diet_supply_year_yield_pop_ref2['Years'].join(mappings['Yield category']).groupby('Yield category').sum()
diet_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

#waste_supply_year_yield_pop_ref2_yc = waste_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
waste_supply_year_yield_pop_ref2_yc = waste_supply_year_yield_pop_ref2['Years'].join(mappings['Yield category']).groupby('Yield category').sum()
waste_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

#pds_supply_year_yield_pop_ref2_yc = pds_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
pds_supply_year_yield_pop_ref2_yc = pds_supply_year_yield_pop_ref2['Years'].join(mappings['Yield category']).groupby('Yield category').sum()
pds_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

idx_vals = set(REF_supply_year_yield_pop_ref1_yc.index.to_list())

for report_category, yield_categories in report_dict.items():
    REF_supply_year_yield_pop_ref1_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    REF_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    diet_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    waste_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    pds_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    
# why are each of these identical?
supply_results = {}

ref1_series = REF_supply_year_yield_pop_ref1_yc.groupby('Report Category').sum() / 10**6
ref1_series.loc['Meat'] += ref1_series.loc['Milk and egg']
ref1_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

ref2_series = REF_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
ref2_series.loc['Meat'] += ref2_series.loc['Milk and egg']
ref2_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)
 
diet_series = diet_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
diet_series.loc['Meat'] += diet_series.loc['Milk and egg']
diet_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

waste_series = waste_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
waste_series.loc['Meat'] += waste_series.loc['Milk and egg']
waste_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

pds_series = pds_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
pds_series.loc['Meat'] += pds_series.loc['Milk and egg']
pds_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)


In [61]:
# Supply_ADOPTION_DATE

ref1_ref2_diff = ref1_series.loc[:,years] - ref2_series.loc[:,years]
a = ref1_ref2_diff.loc['Grain', '2015':'2060'].sum() / 10**9
b = ref1_ref2_diff.loc['Meat', '2015':'2060'].sum() / 10**9
c = ref1_ref2_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Family Planning'] = (a,b,c)

ref2_diet_diff = ref2_series.loc[:,years] - diet_series.loc[:,years]
d = ref2_diet_diff.loc['Grain', '2015':'2060'].sum() / 10**9
e = ref2_diet_diff.loc['Meat', '2015':'2060'].sum() / 10**9
f = ref2_diet_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Plant-rich Diet only (75%)'] = (d,e,f)

ref2_waste_diff = ref2_series.loc[:,years] - waste_series.loc[:,years]
g = ref2_waste_diff.loc['Grain', '2015':'2060'].sum() / 10**9  
h = ref2_waste_diff.loc['Meat', '2015':'2060'].sum() / 10**9
i = ref2_waste_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Food waste only (75%)'] = (g,h,i)

ref2_pds_diff = ref2_series.loc[:,years] - pds_series.loc[:,years]
j = ref2_pds_diff.loc['Grain', '2015':'2060'].sum() / 10**9
k = ref2_pds_diff.loc['Meat', '2015':'2060'].sum() / 10**9
m = ref2_pds_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Integrated Food Solutions (Diet + Waste)'] = (j,k,m)

supply_results['Percent of yield reductions from diet soln'] = (100*d/j, 100*e/k, 100*f/m)
supply_results['Percent of yield reductions from waste soln'] = (100*(1- d/j),100*(1- e/k),100*(1- f/m))

report = pd.DataFrame(supply_results).transpose()
report.columns = ('crop-based reduction','livestock reduction','seafood reduction')
report


,crop-based reduction,livestock reduction,seafood reduction
Family Planning,16.249240,4.341611,0.587320
Plant-rich Diet only (75%),19.423779,10.847489,1.537599
Food waste only (75%),49.488641,6.019456,1.481679
Integrated Food Solutions (Diet + Waste),65.671938,15.690184,2.751573
Percent of yield reductions from diet soln,29.576985,69.135514,55.880731
Percent of yield reductions from waste soln,70.423015,30.864486,44.119269


In [62]:
# Yield_ADOPTION_DATE
yield_results = {}
ref1_series = REF_supply_year_yield_pop_ref1['Years'].sum()
ref2_series = REF_supply_year_yield_pop_ref2['Years'].sum()
diet_series = diet_supply_year_yield_pop_ref2['Years'].sum()
waste_series = waste_supply_year_yield_pop_ref2['Years'].sum()
pds_series = pds_supply_year_yield_pop_ref2['Years'].sum()

ref1_ref2_diff = ref1_series - ref2_series
a = ref1_ref2_diff.loc['2020':'2050'].sum() / 10**15
b = ref1_ref2_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Family Planning'] = (a,b)

ref2_diet_diff = ref2_series - diet_series
c = ref2_diet_diff.loc['2020':'2050'].sum() / 10**15
d = ref2_diet_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Plant-rich Diet only (75%)'] = (c,d)

ref2_waste_diff = ref2_series - waste_series
e = ref2_waste_diff.loc['2020':'2050'].sum() / 10**15
f = ref2_waste_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Food waste only (75%)'] = (e,f)

ref2_pds_diff = ref2_series - pds_series
g = ref2_pds_diff.loc['2020':'2050'].sum() / 10**15
h = ref2_pds_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Integrated Food Solutions (Diet + Waste)'] = (g,h)

yield_results['Percent of yield reductions from diet soln'] = (100*c/g, 100*d/h)
yield_results['Percent of yield reductions from waste soln'] = (100*(1- c/g),100*(1- d/h))

report = pd.DataFrame(yield_results).transpose()
report.columns = ('2020-50 cumulative reduction','2015-60 cumulative reduction')
report



,2020-50 cumulative reduction,2015-60 cumulative reduction
Family Planning,10.427293,21.178171
Plant-rich Diet only (75%),17.488145,31.808868
Food waste only (75%),35.261917,56.989776
Integrated Food Solutions (Diet + Waste),50.603148,84.113695
Percent of yield reductions from diet soln,34.559402,37.816514
Percent of yield reductions from waste soln,65.440598,62.183486


In [63]:
# Emissions_ADOPTION_DATE

emissions_results = {}
ref1_series = REF1_emissions['Years'].sum()
ref2_series = REF2_emissions['Years'].sum()
diet_series = diet_emissions['Years'].sum()
waste_series = waste_emissions['Years'].sum()
pds_series = pds_emissions['Years'].sum()

ref1_ref2_diff = ref1_series.loc[years] - ref2_series.loc[years]
a = ref1_ref2_diff.loc['2020':'2050'].sum() / 10**9
b = ref1_ref2_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Family Planning'] = (a,b)

ref2_diet_diff = ref2_series.loc[years] - diet_series.loc[years]
c = ref2_diet_diff.loc['2020':'2050'].sum() / 10**9
d = ref2_diet_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Plant-rich Diet only (75%)'] = (c,d)

ref2_waste_diff = ref2_series.loc[years] - waste_series.loc[years]
e = ref2_waste_diff.loc['2020':'2050'].sum() / 10**9
f = ref2_waste_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Food waste only (75%)'] = (e,f)

ref2_pds_diff = ref2_series.loc[years] - pds_series.loc[years]
g = ref2_pds_diff.loc['2020':'2050'].sum() / 10**9
h = ref2_pds_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Integrated Food Solutions (Diet + Waste)'] = (g,h)

emissions_results['Percent of yield reductions from diet soln'] = (100*c/g, 100*d/h)
emissions_results['Percent of yield reductions from waste soln'] = (100*(1- c/g),100*(1- d/h))

report = pd.DataFrame(emissions_results).transpose()
report.columns = ('2020-50 cumulative reduction','2015-60 cumulative reduction')
report

,2020-50 cumulative reduction,2015-60 cumulative reduction
Family Planning,21.600129,44.335328
Plant-rich Diet only (75%),86.879029,152.892116
Food waste only (75%),47.162057,77.172427
Integrated Food Solutions (Diet + Waste),124.925683,211.553377
Percent of yield reductions from diet soln,69.544570,72.271177
Percent of yield reductions from waste soln,30.455430,27.728823


In [64]:
#The datasets calculated below match the tables on their respective worksheet tabs in the s/sht.
# Note that the s/sht tables are not updated properly. S/sht formulas need to be copied down and recalculated.

#REF_country_supply
REF_supply_year_yield_pop_ref1['Years'].groupby('Country').sum()/10**6
REF_supply_year_yield_pop_ref2['Years'].groupby('Country').sum()/10**6

#PDS_country_supply
diet_supply_year_yield_pop_ref2['Years'].groupby('Country').sum()/10**6
waste_supply_year_yield_pop_ref2['Years'].groupby('Country').sum()/10**6
pds_supply_year_yield_pop_ref2['Years'].groupby('Country').sum()/10**6

# REF_commodity_supply
REF_supply_year_yield_pop_ref1['Years'].groupby('FAO commodity').sum()/10**6
REF_supply_year_yield_pop_ref2['Years'].groupby('FAO commodity').sum()/10**6

diet_supply_year_yield_pop_ref2['Years'].groupby('FAO commodity').sum()/10**6
waste_supply_year_yield_pop_ref2['Years'].groupby('FAO commodity').sum()/10**6
pds_supply_year_yield_pop_ref2['Years'].groupby('FAO commodity').sum()/10**6


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
FAO commodity,,,,,,,,,,,,,,,,,,,,,
Apples and products,89020906.796523,87865297.123609,92374332.158625,95699631.036791,99038375.548954,98774643.570476,98551870.60107,98162407.497082,97718675.822519,97225591.408048,...,59223763.306434,58875655.451631,58526948.804278,58177421.132232,57826813.758007,57474829.926455,57121147.327315,56765411.942844,56407225.629699,56046163.961052
"Aquatic Animals, Others",1131836.245081,1137850.273684,1149388.822478,1055462.016219,1060435.154493,1028432.13352,1008048.013645,985922.635568,963088.598726,939642.89531,...,172566.911511,170896.925597,169232.173046,167569.94597,165907.219696,164240.808367,162567.377872,160883.501922,159185.625128,157470.073155
Aquatic Plants,1653594.81967,1802591.973551,2022487.104521,2600860.784517,2604084.39412,2541134.424858,2477999.304219,2412057.744057,2345507.819113,2278468.627499,...,326641.103433,322499.124432,318386.433542,314313.758824,310292.569221,306332.604902,302443.860049,298634.032966,294910.557782,291279.149632
Bananas,129765872.86753,137637987.461232,131838771.899113,133626590.515075,134431734.709459,135722166.194863,137068298.041967,138290710.243134,139466262.900657,140594464.377916,...,168398297.058966,168623897.049557,168838663.826786,169042978.554907,169237145.099224,169421424.221445,169595995.133424,169760953.522199,169916317.058416,170062017.291077
Barley and products,8813372.962349,9008331.561868,8323637.922437,8486125.774194,9306298.322232,9368764.830937,9428860.666007,9483078.627936,9534165.185224,9582362.923471,...,11107097.109212,11105930.988327,11103504.086194,11099815.492198,11094859.244115,11088630.087147,11081121.430796,11072324.572542,11062229.739006,11050825.857975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tomatoes and products,202465798.891797,206021568.329912,204233933.640795,210260080.724758,213516977.675247,213595247.764545,213648033.49014,213405680.759234,213073213.270788,212659186.902011,...,180998790.053266,180957840.0293,180906308.514703,180843939.662051,180770402.645529,180685300.829448,180588173.700461,180478478.152626,180355584.020069,180218786.415764
"Vegetables, Other",1010684059.752482,1037225365.153331,1058907830.379421,1072495415.672857,1091620824.92434,1091028263.75178,1092079739.500584,1091613707.295468,1090465396.634428,1088685942.140242,...,785972241.820218,783271190.561358,780555124.3433,777821390.203032,775066840.01651,772287918.886599,769480732.179207,766640997.81178,763764020.296484,760844718.89926
Wheat and products,566511695.959862,573788959.514749,585479568.758057,594914332.39808,599545551.421892,596975435.302906,594753905.212875,591916459.280956,588907310.430267,585745825.689085,...,412677201.248175,411202955.354404,409709038.087463,408195495.645125,406662161.290802,405108650.400764,403534363.157614,401938441.273263,400319738.484287,398676880.220537
